# Análise das Intents

Este jupyter-notebook vai auxiliar na análise de detecção de intenções de seu chatbot.

### Configurando e Imports

In [1]:
from IPython.display import IFrame

import rasa
print("Rasa: {}".format(rasa.__version__))

Rasa: 2.8.7


### Treinando o modelo do Rasa NLU

* Para avaliar o bot o primeiro passo é treinar o seu chatbot. Mas não é necessário treinar a parte de conversação completa (rasa_core) apenas a parte de interpretação de mensagens (rasa_nlu).

* O comando `train-nlu` do Makefile executa o treinamento apenas do `rasa_nlu`

In [2]:
!rasa train nlu --config $CONFIG_PATH --nlu $DATA_PATH

Training NLU model...
2021-09-28 20:36:05 INFO     rasa.shared.nlu.training_data.training_data  - Training data stats:
2021-09-28 20:36:05 INFO     rasa.shared.nlu.training_data.training_data  - Number of intent examples: 298 (25 distinct intents)

2021-09-28 20:36:05 INFO     rasa.shared.nlu.training_data.training_data  -   Found intents: 'relacionamento', 'time', 'religiao', 'cor', 'onde_voce_mora', 'tudo_bem', 'despedir', 'menu', 'cumprimentar', 'pede_exemplo_form', 'historia', 'negar', 'comida', 'licenca', 'cancelar', 'afirmar', 'pede_exemplo_acao', 'signo', 'piada', 'informar', 'triste', 'me_ajuda', 'musica', 'genero', 'elogios'
2021-09-28 20:36:05 INFO     rasa.shared.nlu.training_data.training_data  - Number of response examples: 0 (0 distinct responses)
2021-09-28 20:36:05 INFO     rasa.shared.nlu.training_data.training_data  - Number of entity examples: 173 (20 distinct entities)
2021-09-28 20:36:05 INFO     rasa.shared.nlu.training_data.training_data  -   Found entity types: 

O comando acima gerou um modelo apenas de **NLU**, este modelo pode ser achado na seguinte pasta:

`modules/notebooks/intents/models/`

In [3]:
!ls models/

nlu-20210928-203038.tar.gz  nlu-20210928-203758.tar.gz


### Métodos de avaliação do chatbot

* O Rasa fornece vários métodos de avaliação e validação das `intents`, para verificar como utilizá-los, cada método fornece um log, imagem, gráfico ou arquivo com dados relevantes para interpretação do chatbot.

* O comando `rasa test nlu` é a base para a avaliação do chatbot.

* Na célula abaixo a flag `-h` foi utilizada para mostrar as funções e a forma de uso de cada uma delas, mude seus valores e flags para ter as informações desejadas na sua análise.

* Atualmente o Rasa possui 2 modos, `evaluation` e `crossvalidation` que tem seções dedicadas a eles neste jupyter notebook.

In [4]:
!rasa test nlu -h

usage: rasa test nlu [-h] [-v] [-vv] [--quiet] [-m MODEL] [-u NLU] [--out OUT]
                     [-c CONFIG [CONFIG ...]] [--cross-validation] [-f FOLDS]
                     [-r RUNS] [-p PERCENTAGES [PERCENTAGES ...]] [--no-plot]
                     [--successes] [--no-errors] [--no-warnings]

optional arguments:
  -h, --help            show this help message and exit
  -m MODEL, --model MODEL
                        Path to a trained Rasa model. If a directory is
                        specified, it will use the latest model in this
                        directory. (default: models)
  -u NLU, --nlu NLU     File or folder containing your NLU data. (default:
                        data)
  --out OUT             Output path for any files created during the
                        evaluation. (default: results)
  -c CONFIG [CONFIG ...], --config CONFIG [CONFIG ...]
                        Model configuration file. If a single file is passed
                        and cross valid

### Evaluation

* O comando abaixo gera informações relevates para a validação das `intents` são elas:
    * Matriz de confusão
    * Histograma
    * Erros de intents

Essas informações e os demais arquivos podem ser encontrados na pasta `results`.

In [5]:
!rm -f results/*erros.json
!cd /bot && rasa test nlu --nlu $DATA_PATH --model $MODELS_PATH

/usr/local/lib/python3.7/site-packages/rasa/shared/core/domain.py:2006: FutureWarning: The definition of slot mappings in your form should always be preceded by the keyword `required_slots`. The lack of this keyword will be deprecated in Rasa Open Source 3.0.0. Please see https://rasa.com/docs/rasa/forms for more information.
  f"The definition of slot mappings in your form "
2021-09-28 20:38:11 INFO     rasa.model  - Loading model models/20210928-172150.tar.gz...
/usr/local/lib/python3.7/site-packages/rasa/shared/core/domain.py:2006: FutureWarning: The definition of slot mappings in your form should always be preceded by the keyword `required_slots`. The lack of this keyword will be deprecated in Rasa Open Source 3.0.0. Please see https://rasa.com/docs/rasa/forms for more information.
  f"The definition of slot mappings in your form "
2021-09-28 20:38:23 INFO     rasa.nlu.test  - Running model for predictions:
100%|████████████████████████████████████████| 298/298 [00:02<00:00, 108.90

In [6]:
!ls results

DIETClassifier_confusion_matrix.png  intent_confusion_matrix.png
DIETClassifier_errors.json	     intent_histogram.png
DIETClassifier_histogram.png	     intent_report.json
DIETClassifier_report.json


#### Matriz de Confusão
* A matriz de confusão mostra a correlação entre as intents.
* A diagonal principal tem forte correlação pois mostra a relação de uma intent **com ela mesma**
* O ideal é que não haja **nenhum valor** diferente de **0 fora da diagonal principal**.

In [7]:
IFrame(src='./results/intent_confusion_matrix.png', width=900, height=700)

#### Erros
* O arquivo `intent_errors.json` mostra os erros encontrados nas intents. Este arquivo lista os mesmos erros mostrados na **matriz de confusão**, então caso ela fique muito grande, não se preocupe, pois você pode apenas procurar os erros no arquivo gerado.
* Os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.
* Caso o arquivo não seja gerado significa que não foram encontrados erros.
* O teste pode encontrar erros específicos de componentes do pipeline. Procure por arquivos cujo final seja `_errors.json` para ver todos.

In [8]:
!ls results/

DIETClassifier_confusion_matrix.png  intent_confusion_matrix.png
DIETClassifier_errors.json	     intent_histogram.png
DIETClassifier_histogram.png	     intent_report.json
DIETClassifier_report.json


In [9]:
!cat results/intent_errors.json

cat: results/intent_errors.json: No such file or directory


In [10]:
!cat results/DIETClassifier_errors.json

[
  {
    "text": "você tem namorado",
    "entities": [],
    "predicted_entities": [
      {
        "entity": "relationship",
        "start": 9,
        "end": 17,
        "confidence_entity": 0.5268345475196838,
        "value": "namorado",
        "extractor": "DIETClassifier"
      }
    ]
  }
]

#### Histograma

* O histograma contém a distribuição da predições das `intents`

In [11]:
IFrame(src='./results/intent_histogram.png', width=900, height=700)

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `intents` são:

* [Evaluation](https://rasa.com/docs/nlu/evaluation/)